<img src="./images/DLI_Header.png" style="width: 400px;">

# 使用预训练的模型

虽然通常必须要有一个标注良好的大型数据集才能训练出一个好的深度学习模型，但也有很多“开箱即用”的免费预训练模型供我们使用。在您决定开展自己的深度学习项目时，建议您先从在线查找能够帮助您实现目标的现有模型开始。[Model Zoo](https://modelzoo.co/) 是一个提供可用模型的出色平台。GitHub 上也有很多模型，您可以通过在网上搜索来查找这些模型。

## 目标

完成本节内容的学习后，您将能够：
* 使用 Keras 加载经过良好训练的预训练模型
* 对您自己的图像进行预处理，使其适用于预训练模型
* 使用预训练模型对您自己的图像进行精确推理

## 自动化的狗门

在本练习中，我们将创建一个只允许狗进出的狗门，而将猫留在屋内，其他动物则只能在屋外待在它们应该待的地方。通常，我们需要一个非常大的数据集，其中要包含很多狗以及其他动物的图片。所幸，我们可以找到现成的模型，它们已被包含大量动物图片的大型数据集训练过了。

[ImageNet](https://en.wikipedia.org/wiki/ImageNet#History_of_the_ImageNet_challenge) 挑战赛已产生许多可用于图像分类的顶尖模型。这些模型均使用数百万张图像进行了训练，能够准确地将图像分类为 1000 种不同的物体，其中很多类别属于动物，包括多个品种的狗和猫。这些都是适用于狗门的理想模型。

## 加载模型

我们先来下载模型。经过训练的 ImageNet 模型可以直接从 Keras 库中下载。您可在 [此处](https://keras.io/api/applications/#available-models) 看到可用的模型及其详情。其中任何模型都适用于我们的练习。我们将选取常用模型 [VGG16](https://keras.io/api/applications/vgg/)：

In [ ]:
from tensorflow.keras.applications import VGG16
  
# load the VGG16 network *pre-trained* on the ImageNet dataset
model = VGG16(weights="imagenet")

此模型现已加载完毕，我们来观察一下。您会注意到，它看起来很像手语练习中的卷积模型。请注意第一层（输入层）和最后一层（输出层）。和先前的练习一样，我们需要确保图像与模型预期的输入维度相匹配。另外，了解模型将从最后的输出层中返回何种结果也非常有用。

In [ ]:
model.summary()

### 输入维度
我们可以看到，模型预期的图像形状为 (224, 224, 3)，对应于 224 像素的高度、224 像素的宽度和 3 个颜色通道。正如我们在上一个练习中了解的那样，Keras 模型一次可接受多个图像来进行预测。如果我们只传入一个图像，该图像的形状将为 (1, 224, 224, 3)。我们需要确保，在向模型传入图像以供预测时，图像要与这些维度相匹配。

### 输出维度
我们还可看到，模型将返回形状为 1000 的预测。要记得，在第一个练习中，模型的输出形状为 10，对应于 10 个不同数字。在第二个练习中，形状为 24，对应于手语字母表中的 24 个字母，而这些字母可以在静止图像中捕获。而在本练习中，该图像将放入 1000 种可能的类别中。虽然完整的 ImageNet 数据集包含 20000 多个类别，但该竞赛以及最终产生的预训练模型仅会使用其中 1000 个类别。我们可以在此处看一下所有 [可能的类别](https://gist.github.com/yrevar/942d3a0ac09ec9e5eb3a)。

其中很多类别均为动物，包括各个种类的狗和猫。狗的类别范围是 151 到 268。猫的类别范围是 281 到 285。我们将能使用这些类别告知狗门哪类动物在门口，以及我们是否应该让它进来。

## 加载图像
和先前练习中的操作一样，我们首先加载图像并将其显示出来。

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

def show_image(image_path):
    image = mpimg.imread(image_path)
    print(image.shape)
    plt.imshow(image)

In [ ]:
show_image("doggy_door_images/happy_dog.jpg")

## 预处理图像

接下来，我们需要对图像进行预处理，以便能以适当的格式将其送入模型中。这与我们在上一个练习中预测手语图像时的操作类似。要记得，本例中的图像的最终形状需为 (1, 224, 224, 3)。

如果直接从 Keras 加载模型，我们还可利用 [`preprocess_input`方法](https://www.tensorflow.org/api_docs/python/tf/keras/applications/vgg16/preprocess_input)。通过结合这些方法和特定模型，用户可以对自己的图像进行预处理，使其达到最初训练模型时所采用的图像的质量。在对新的 ASL 图像进行推理时，您必须自己手动执行此操作：

In [ ]:
from tensorflow.keras.preprocessing import image as image_utils
from tensorflow.keras.applications.vgg16 import preprocess_input

def load_and_process_image(image_path):
    # Print image's original shape, for reference
    print('Original image shape: ', mpimg.imread(image_path).shape)
    
    # Load in the image with a target size of 224,224
    image = image_utils.load_img(image_path, target_size=(224, 224))
    # Convert the image from a PIL format to a numpy array
    image = image_utils.img_to_array(image)
    # Add a dimension for number of images, in our case 1
    image = image.reshape(1,224,224,3)
    # Preprocess image to align with original ImageNet dataset
    image = preprocess_input(image)
    # Print image's shape after processing
    print('Processed image shape: ', image.shape)
    return image

In [ ]:
processed_image = load_and_process_image("doggy_door_images/brown_bear.jpg")

## 进行预测

现在我们已将图像处理成正确的格式，接着我们就可将其传入模型并获取预测。我们预期会输出一个包含 1000 个元素的数组，这读起来不太容易。所幸，直接从 Keras 加载的模型还可采用另一种有用的方法，将预测数组转换为可读性更高的形式。

填写下列函数以实现预测：

In [ ]:
from tensorflow.keras.applications.vgg16 import decode_predictions

def readable_prediction(image_path):
    # Show image
    FIXME
    # Load and pre-process image
    image = FIXME
    # Make predictions
    predictions = FIXME
    # Print predictions in readable form
    print('Predicted:', decode_predictions(predictions, top=3))

### 答案

单击下方的“...”查看答案。

```python
from tensorflow.keras.applications.vgg16 import decode_predictions

def readable_prediction(image_path):
    # Show image
    show_image(image_path)
    # Load and pre-process image
    image = load_and_process_image(image_path)
    # Make predictions
    predictions = model.predict(image)
    # Print predictions in readable form
    print('Predicted:', decode_predictions(predictions, top=3))
```

请尝试对一些动物进行预测，并查看结果！另外，您还可随时上传自己的图像并对它们进行分类。您可能会对此模型的出色程度感到吃惊。

In [ ]:
readable_prediction("doggy_door_images/happy_dog.jpg")

In [ ]:
readable_prediction("doggy_door_images/brown_bear.jpg")

In [ ]:
readable_prediction("doggy_door_images/sleepy_cat.jpg")

## 仅允许狗进出

为了仅允许狗进出并将猫留在屋内，在利用模型进行预测时，我们需要选择合适的类别。狗的类别范围是 151 到 268，而猫的类别范围是 281 到 285。您可以使用 [`np.argmax`](https://numpy.org/doc/stable/reference/generated/numpy.argmax.html) 函数确定预测结果的数组中哪个类别具有最大的概率值。如果需要帮助，可单击下方的三个点来查看答案。

In [ ]:
import numpy as np

def doggy_door(image_path):
    show_image(image_path)
    image = load_and_process_image(image_path)
    preds = model.predict(image)
    if FIXME:
        print("Doggy come on in!")
    elif FIXME:
        print("Kitty stay inside!")
    else:
        print("You're not a dog! Stay outside!")

### 答案

单击下方的“...”查看答案。

```python
import numpy as np

def doggy_door(image_path):
    show_image(image_path)
    image = load_and_process_image(image_path)
    preds = model.predict(image)
    if 151 <= np.argmax(preds) <= 268:
        print("Doggy come on in!")
    elif 281 <= np.argmax(preds) <= 285:
        print("Kitty stay inside!")
    else:
        print("You're not a dog! Stay outside!")
```

In [ ]:
doggy_door("doggy_door_images/brown_bear.jpg")

In [ ]:
doggy_door("doggy_door_images/happy_dog.jpg")

In [ ]:
doggy_door("doggy_door_images/sleepy_cat.jpg")

## 总结

您做得很棒！通过使用强大的预训练模型，我们只用几行代码便创建出了实用的狗门。令人兴奋的是，我们无需做大量的前期工作即可利用深度学习。而且，随着深度学习社区的不断发展，您还可将更多模型用在自己的项目中。

### 清理显存
继续进行后续内容之前，请执行以下单元清理 GPU 显存。

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

## 下一步

预训练模型固然非常强大，但其有时也会无法完全适配您的数据。下一节中，您将学习另一项强大技术“*迁移学习*”，它可以帮助您定制预训练模型，进而对您的数据作出良好预测。

继续学习下一节：[*预训练模型*](./05b_presidential_doggy_door.ipynb)。